In [ ]:
left_link_attempt_flag = False
right_link_attempt_flag = False
left_link_duration = 0
right_link_duration = 0

left_link_p = 0.1
right_link_p = 0.1

left_link_attempt_flag = False
right_link_attempt_flag = False
left_link_finishing_time = 0
right_link_finishing_time = 0
left_link_half_finishing_time = 0
right_link_half_finishing_time = 0
left_link_half_arrived_flag = False
right_link_half_arrived_flag = False
left_link_half_success_flag = False
right_link_half_success_flag = False
left_link_half_failed_flag = False
right_link_half_failed_flag = False
repeater_swap_flag = False
swap_arrived_flag = False
left_qubit_expired_flag = False
right_qubit_expired_flag = False



In [ ]:
def update(clock_counter):
    if  (not left_memory_object.attempt_flag and 
        not left_node_memory_exist_flag and 
        not right_node_memory_exist_flag and not left_memory_object.generated_flag):
        left_memory_object.attempt_flag = True
        left_memory_object.entangled_node = link[1]
        right_memory_object.entangled_node = link[0]
        left_memory_object.attempt_start_time = current_clock_counter
        left_memory_object.EPR_age = -1
        right_memory_object.EPR_age = -1
        left_memory_object.expired = False
        right_memory_object.expired = False
        left_memory_object.arrived_on_other_side_flag = False
        left_memory_object.arrived_on_both_sides_flag = False
        right_memory_object.arrived_on_other_side_flag = False
        right_memory_object.arrived_on_both_sides_flag = False


        if self.experimenting_classical_communication:

            left_memory_object.attempt_finishing_time = current_clock_counter+2*left_memory_object.attached_link_duration+left_memory_object.added_clock
            left_memory_object.attempt_arriving_time_other_side=current_clock_counter+1*left_memory_object.attached_link_duration+left_memory_object.added_clock
        else:
            left_memory_object.attempt_finishing_time = current_clock_counter+1
        
            

            
            
        left_memory_object.memory_last_updated_clock  = current_clock_counter
        right_memory_object.memory_last_updated_clock = current_clock_counter

                
    elif left_memory_object.attempt_flag:#we have attempted before lets check if it is done

        if left_memory_object.memory_last_updated_clock<current_clock_counter or current_clock_counter==0:
            if self.each_repeater_left_right_memory_age_tracking_flag[self.path_id,link[0],"right",memory_id]:
                left_memory_object.EPR_age = left_memory_object.EPR_age+self.time_granularity_value
                left_memory_object.memory_last_updated_clock  = current_clock_counter
        if right_memory_object.memory_last_updated_clock<current_clock_counter or current_clock_counter==0:
            if self.each_repeater_left_right_memory_age_tracking_flag[self.path_id,link[1],"left",memory_id]:
                right_memory_object.EPR_age= right_memory_object.EPR_age+self.time_granularity_value
                right_memory_object.memory_last_updated_clock = current_clock_counter
        EPR_age = left_memory_object.EPR_age
        right_EPR_age = right_memory_object.EPR_age
        if (EPR_age >self.each_link_cut_off[link] and self.having_cut_offs and 
            link[0] not in self.each_path_source_destination[self.path_id]) or (
            right_EPR_age >self.each_link_cut_off[link] and self.having_cut_offs and 
            link[1] not in self.each_path_source_destination[self.path_id]):
                expired_flag = True
                self.expired_qubits_counter+=1
                self.last_generated_expiration_message = current_clock_counter
                

                left_memory_object.attempt_flag = False
                left_memory_object.expired = True
                left_memory_object.EPR_age = -1
                right_memory_object.EPR_age = -1

                """we send a message to the source node to execute these commands locally there. 
                We need to have a local version of this for links to know when they can attempt"""
                self.each_repeater_left_right_memory_exist_flag[self.path_id,link[0],"right",memory_id] = False
                self.each_repeater_left_right_memory_exist_flag[self.path_id,link[1],"left",memory_id]  = False
                self.each_repeater_left_right_memory_age_tracking_flag[self.path_id,link[0],"right",memory_id]  = False
                self.each_repeater_left_right_memory_age_tracking_flag[self.path_id,link[1],"left",memory_id]  = False

    
    
        elif left_memory_object.attempt_arriving_time_other_side <= current_clock_counter and not left_memory_object.arrived_on_other_side_flag:
            random_value = random.uniform(0,1)
            left_memory_object.arrived_on_other_side_flag = True
            if random_value<= left_memory_object.attempt_success_p and left_memory_object.EPR_age+left_memory_object.attached_link_duration<self.each_link_cut_off[link]:
                self.each_repeater_left_right_memory_exist_flag[self.path_id,link[1],"left",memory_id] = True
                self.each_repeater_left_right_memory_age_tracking_flag[self.path_id,link[1],"left",memory_id]  = True

                photon_arrived_at_other_side = True
                right_memory_object.EPR_age=-1

                left_memory_object.attempt_result= True
                right_memory_object.generated_flag = True
                right_memory_object.qubit_storing_time = current_clock_counter
                self.last_generated = current_clock_counter+left_memory_object.attached_link_duration
            else:#
                left_memory_object.attempt_result= False
                photon_arrived_at_other_side = False
                left_memory_object.EPR_age = -1
                right_memory_object.EPR_age=-1

                self.each_repeater_left_right_memory_exist_flag[self.path_id,link[0],"right",memory_id] = False
                self.each_repeater_left_right_memory_exist_flag[self.path_id,link[1],"left",memory_id]  = False
                self.each_repeater_left_right_memory_age_tracking_flag[self.path_id,link[0],"right",memory_id]= False
                self.each_repeater_left_right_memory_age_tracking_flag[self.path_id,link[1],"left",memory_id]= False
                self.last_generated = current_clock_counter+left_memory_object.attached_link_duration
                left_memory_object.generated_flag = False
                right_memory_object.generated_flag = False
        elif left_memory_object.attempt_finishing_time <= current_clock_counter and not left_memory_object.arrived_on_both_sides_flag:
            left_memory_object.arrived_on_both_sides_flag = True
            if left_memory_object.attempt_result and not left_memory_object.expired:
                left_memory_object.attempt_flag = False
                left_memory_object.generated_flag = True
                left_memory_object.attempt_result = False
                self.last_generated = current_clock_counter
                if not self.entanglement_generation_delay:
                    left_memory_object.EPR_age = -1
                    right_memory_object.EPR_age = -1
                left_memory_object.fidelity = self.initial_fidelity
                self.each_repeater_left_right_memory_exist_flag[self.path_id,link[0],"right",memory_id] = True
                self.each_repeater_left_right_memory_exist_flag[self.path_id,link[1],"left",memory_id]  = True
            else:
                self.last_generated = current_clock_counter
                self.failed_at_least_one_link = True
                left_memory_object.attempt_flag = False
                left_memory_object.generated_flag = False
                right_memory_object.generated_flag = False

                left_memory_object.EPR_age = -1
                right_memory_object.EPR_age = -1
    else:
        if left_memory_object.memory_last_updated_clock<current_clock_counter or current_clock_counter==0:
            if self.each_repeater_left_right_memory_age_tracking_flag[self.path_id,link[0],"right",memory_id]:
                left_memory_object.EPR_age = left_memory_object.EPR_age+self.time_granularity_value
                left_memory_object.memory_last_updated_clock  = current_clock_counter
        if right_memory_object.memory_last_updated_clock<current_clock_counter or current_clock_counter==0:
            if self.each_repeater_left_right_memory_age_tracking_flag[self.path_id,link[1],"left",memory_id]:
                right_memory_object.EPR_age= right_memory_object.EPR_age+self.time_granularity_value
                right_memory_object.memory_last_updated_clock = current_clock_counter
        EPR_age = left_memory_object.EPR_age
        right_EPR_age = right_memory_object.EPR_age
        if (EPR_age >self.each_link_cut_off[link] and self.having_cut_offs and 
            link[0] not in self.each_path_source_destination[self.path_id]) or (
            right_EPR_age >self.each_link_cut_off[link] and self.having_cut_offs and 
            link[1] not in self.each_path_source_destination[self.path_id]):
            self.expired_qubits_counter+=1
            left_memory_object.expired = True
            
            expired_flag =True
            self.last_generated_expiration_message = current_clock_counter
            
            left_memory_object.attempt_flag = False
            left_memory_object.EPR_age = -1
            right_memory_object.EPR_age = -1
            self.each_repeater_left_right_memory_exist_flag[self.path_id,link[0],"right",memory_id] = False
            self.each_repeater_left_right_memory_exist_flag[self.path_id,link[1],"left",memory_id]  = False
            self.each_repeater_left_right_memory_age_tracking_flag[self.path_id,link[0],"right",memory_id]  = False
            self.each_repeater_left_right_memory_age_tracking_flag[self.path_id,link[1],"left",memory_id]  = False
    

    if left_memory_object.memory_last_updated_clock<current_clock_counter or current_clock_counter==0:
            left_memory_object.EPR_age = left_memory_object.EPR_age+self.time_granularity_value
            left_memory_object.memory_last_updated_clock  = current_clock_counter
    if right_memory_object.memory_last_updated_clock<current_clock_counter or current_clock_counter==0:
        if self.each_repeater_left_right_memory_age_tracking_flag[self.path_id,link[1],"left",memory_id]:
            right_memory_object.EPR_age= right_memory_object.EPR_age+self.time_granularity_value
            right_memory_object.memory_last_updated_clock = current_clock_counter

            

In [ ]:
for clock_counter in range(0,end_time,time_granularity_value):
    update(clock_counter) 
    swap_operation(clock_counter)  
    check_swap_results_arrived(clock_counter)
    if all_are_generated():
        if none_expired():
            swap_operation(clock_counter)
            check_swap_results_arrived(clock_counter)
        else:
            self.check_swap_results_arrived(clock_counter)
            self.check_all_swaps(clock_counter)
            if last_generated_expiration_message+e2e_communication_duration <= clock_counter:
                left_memory_object  = self.each_path_repeater_left_right_memory_id_memory_object[self.path_id,link[0],"right",memory_id]
                left_memory_object.generated_flag = False
                left_memory_object.expired = False
                self.make_all_memories_free()
                self.all_generated_flag = False
                self.swap_operation_is_performed_flag = False
                self.last_generated_expiration_message = clock_counter
    else:
        if self.none_expired():
            self.check_swap_results_arrived(clock_counter)
            self.check_all_swaps(clock_counter)
        else:
            if self.last_generated_expiration_message+self.e2e_communication_duration <= clock_counter:
                left_memory_object.generated_flag = False
                left_memory_object.expired = False
                self.each_path_swap_on_source_node_qubit_arrived[self.path_id]=False
                self.each_path_swap_on_source_node_qubit_id[self.path_id] = -1
                self.each_path_swap_on_end_node_qubit_id[self.path_id] = -1
                self.make_all_memories_free()
                self.all_generated_flag = False
                self.swap_operation_is_performed_flag = False
                self.one_failed_round_times.append(clock_counter-self.last_failed_time)
                self.last_failed_time = clock_counter
                self.last_delivered_e2e_epr_time = clock_counter
                self.last_generated_expiration_message = clock_counter
                if global_go_to_sleep_flag:
                    time.sleep(1)